In [5]:
import pandas as pd
import numpy as np
import torch
import scipy.misc
import torchvision
from torchvision import utils, transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchsummary import summary
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
import csv


In [7]:
# Reading data to create id to label mapping
data = pd.read_csv("/scratch/tmp/WhaleClassData/train.csv")
label_set = set(data['Id'])
num_of_classes = len(label_set)
print("Number of classes: ", len(label_set))

label_to_id = {}
id_to_label = {}

def create_label_id_maps():
    _id = 0 
    for label in label_set: 
        label_to_id[label] = _id 
        id_to_label[_id] = label
        _id += 1

create_label_id_maps()

Number of classes:  4251


In [8]:
# Dataset class definition 
class WhaleDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.root_dir = root_dir
        if "train" in csv_file:
            self.whales = pd.read_csv(csv_file)
            self.test = 0
        else:
            self.whales = list(os.listdir(self.root_dir))
            self.test = 1
        self.transform = transform
    
    def __len__(self):
        return len(self.whales)
    
    def __getitem__(self, idx):
        if self.test:
            img_name = os.path.join(self.root_dir, self.whales[idx])
        else:
            img_name = os.path.join(self.root_dir, self.whales.iloc[idx, 0])
            
        image = Image.open(img_name).convert('RGB')
        if self.test:
            sample = {'image': image, 'image_name': self.whales[idx]}
        else:
            label = self.whales.iloc[idx, 1]
            label = label_to_id[label]
            sample = {'image': image, 'image_name': self.whales.iloc[idx, 0], 'label': label}
        if self.transform:
            sample['image'] = self.transform(sample['image'])
        return sample

In [9]:
# Defining transformations to aid in data augmentation
resize_shape = (200, 200)
# resize_shape = (700, 1050)

transforms_1 = torchvision.transforms.Compose([
    torchvision.transforms.Resize(resize_shape),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.RandomRotation(20, resample=Image.BILINEAR),
    torchvision.transforms.RandomAffine(30),
    torchvision.transforms.ToTensor()
])

transform_resize = torchvision.transforms.Compose([
    torchvision.transforms.Resize(resize_shape),
    torchvision.transforms.ToTensor()
])

In [10]:
# Creating Train and Valid Datset samplers
whale_dataset_1 = WhaleDataset(csv_file="/scratch/tmp/WhaleClassData/train.csv", 
                             root_dir="/scratch/tmp/WhaleClassData/train",
                             transform=transform_resize)
whale_dataset_2 = WhaleDataset(csv_file="/scratch/tmp/WhaleClassData/train.csv", 
                             root_dir="/scratch/tmp/WhaleClassData/train",
                             transform=transforms_1)
whale_dataset_test = WhaleDataset(csv_file="/scratch/tmp/WhaleClassData/test", 
                             root_dir="/scratch/tmp/WhaleClassData/test/",
                             transform=transform_resize)

concat_dataset = torch.utils.data.ConcatDataset([whale_dataset_1, whale_dataset_2])

validation_split = .2
shuffle_dataset = True
random_seed = 42

dataset_size = len(concat_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [12]:
# Checking what kind of system you are using
try:
  import google.colab
  from google.colab import drive
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False
try:
    hostname = !hostname
    if 'lab' in hostname[0] and '.sci.utah.edu' in hostname[0]:
        IN_CADE = True
    else:
        IN_CADE = False
except:
    IN_CADE = False

assert(not IN_CADE or not IN_COLAB)

#defining the folders where datasets will be, depending on the system
machine_being_used = 'cade' if IN_CADE else ('colab' if IN_COLAB else 'other')
pre_folder = '/scratch/tmp/' if machine_being_used == 'cade' else './'

In [13]:
def define_gpu_to_use(minimum_memory_mb = 3800):
    gpu_to_use = None
    try: 
        os.environ['CUDA_VISIBLE_DEVICES']
        print('GPU already assigned before: ' + str(os.environ['CUDA_VISIBLE_DEVICES']))
        return
    except:
        pass
    torch.cuda.empty_cache()
    for i in range(16):
        free_memory = !nvidia-smi --query-gpu=memory.free -i $i --format=csv,nounits,noheader
        if free_memory[0] == 'No devices were found':
            break
        free_memory = int(free_memory[0])
        if free_memory>minimum_memory_mb-500:
            gpu_to_use = i
            break
    if gpu_to_use is None:
        print('Could not find any GPU available with the required free memory of ' +str(minimum_memory_mb) + 'MB. Please use a different system for this assignment.')
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_to_use)
        print('Chosen GPU: ' + str(gpu_to_use))
        x = torch.rand((256,1024,minimum_memory_mb-500)).cuda()
        x = torch.rand((1,1)).cuda()
        del x

In [14]:
define_gpu_to_use()

Chosen GPU: 0


In [15]:
# Model 1 - Squeezenet - Dataset
batch_size = 128

train_loader = torch.utils.data.DataLoader(concat_dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(concat_dataset, batch_size=1, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(whale_dataset_test, batch_size=1, shuffle=False)

In [ ]:
# def count_params(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# resnet18 = models.resnet18(pretrained=True)
# alexnet = models.alexnet(pretrained=True)
# squeezenet = models.squeezenet1_0(pretrained=True)
# vgg16 = models.vgg16(pretrained=True)
# densenet = models.densenet161(pretrained=True)
# inception = models.inception_v3(pretrained=True)
# googlenet = models.googlenet(pretrained=True)
# shufflenet = models.shufflenetv2(pretrained=True)
# print("resne      ::", count_params(resnet18))
# print("alexn      ::", count_params(alexnet))
# print("squeezenet ::", count_params(squeezenet))
# print("vgg16      ::", count_params(vgg16))
# print("densenet   ::", count_params(densenet))
# print("inception  ::", count_params(inception))
# print("googlenet  ::", count_params(googlenet))
# print("shufflenet ::", count_params(shufflenet))

# param_count_list = [(count_params(resnet18), "resnet18"), 
#                    (count_params(alexnet), "alexnet"),
#                    (count_params(squeezenet), "squeezenet"),
#                    (count_params(vgg16), "vgg16"), 
#                    (count_params(densenet), "densenet"),
#                    (count_params(inception), "inception"),
#                    (count_params(googlenet), "googlenet"),
#                    (count_params(shufflenet), "shufflenet")]

# print(sorted(param_count_list))

In [18]:

squeezenet = models.squeezenet1_0(pretrained=True)
squeezenet

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): Fire(
      (squeeze): Conv2d(128, 32, kerne

In [20]:
squeezenet = squeezenet.cuda()
summary(squeezenet, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 97, 97]          14,208
              ReLU-2           [-1, 96, 97, 97]               0
         MaxPool2d-3           [-1, 96, 48, 48]               0
            Conv2d-4           [-1, 16, 48, 48]           1,552
              ReLU-5           [-1, 16, 48, 48]               0
            Conv2d-6           [-1, 64, 48, 48]           1,088
              ReLU-7           [-1, 64, 48, 48]               0
            Conv2d-8           [-1, 64, 48, 48]           9,280
              ReLU-9           [-1, 64, 48, 48]               0
             Fire-10          [-1, 128, 48, 48]               0
           Conv2d-11           [-1, 16, 48, 48]           2,064
             ReLU-12           [-1, 16, 48, 48]               0
           Conv2d-13           [-1, 64, 48, 48]           1,088
             ReLU-14           [-1, 64,

In [13]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [18]:

model_1 = models.squeezenet1_0(pretrained=True)
model_1.fc = torch.nn.Linear(in_features=1000, out_features=num_of_classes, bias=True)
# model_1.classifier[1] = nn.Conv2d(512, num_of_classes, kernel_size=(1,1), stride=(1,1))
# model_1.classifier[2] = torch.nn.Linear(in_features=1000, out_features=num_of_classes, bias=True)
# model_1.classifier[3] = Identity()

model_1 = model_1.cuda()
# summary(model_1, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 97, 97]          14,208
              ReLU-2           [-1, 96, 97, 97]               0
         MaxPool2d-3           [-1, 96, 48, 48]               0
            Conv2d-4           [-1, 16, 48, 48]           1,552
              ReLU-5           [-1, 16, 48, 48]               0
            Conv2d-6           [-1, 64, 48, 48]           1,088
              ReLU-7           [-1, 64, 48, 48]               0
            Conv2d-8           [-1, 64, 48, 48]           9,280
              ReLU-9           [-1, 64, 48, 48]               0
             Fire-10          [-1, 128, 48, 48]               0
           Conv2d-11           [-1, 16, 48, 48]           2,064
             ReLU-12           [-1, 16, 48, 48]               0
           Conv2d-13           [-1, 64, 48, 48]           1,088
             ReLU-14           [-1, 64,

In [19]:
model_1

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): Fire(
      (squeeze): Conv2d(128, 32, kerne

In [17]:
# Model 1 - squeezenet
num_epochs = 5

model_1 = models.squeezenet1_0(pretrained=True)
# model_1.classifier[1] = nn.Conv2d(512, num_of_classes, kernel_size=(1,1), stride=(1,1))
model_1.fc = torch.nn.Linear(in_features=1000, out_features=num_of_classes, bias=True)
model_1.cuda()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr=0.0001)
   
for epoch in range(num_epochs): 
    model_1.train() 
    losses = []
    print('Epoch ' + str(epoch)) 
    for sample in train_loader:
        optimizer.zero_grad() 
        images = sample['image'].cuda()
        print(images.shape)
        label = sample['label'].cpu().numpy()
        label = label.reshape(label.shape[0],1)
        # Creating one hot vector
        y_onehot = torch.FloatTensor(label.shape[0], num_of_classes)
        y_onehot.zero_()
        y_onehot.scatter_(1, torch.LongTensor(label), 1)
        y_onehot = y_onehot.cuda()
        out = model_1(images)
        loss = criterion(out, y_onehot)
        loss.backward()
        optimizer.step() 
        losses.append(loss.item())
    print('loss: ' + str(np.mean(losses)))

Epoch 0
torch.Size([128, 3, 200, 200])


ValueError: Target size (torch.Size([128, 4251])) must be the same as input size (torch.Size([128, 1000]))

In [ ]:
# writeFile = open('submission.csv', 'w')
# writer = csv.writer(writeFile)
# writer.writerow(["Image", "Id"])

model_1.eval()
counter = 0
acc = 0

with torch.no_grad():
    for sample in validation_loader:
        images = sample['image'].cuda()
#         print (sample['image_name'][0])
#         print (counter)
#         counter += 1
        out = model_1(images)
        out = nn.functional.softmax(out)
        out = out.cpu().detach().numpy()
        ind = np.argsort(out)[0][-5:]
#         ind_last = np.argsort(out)[0][-5:]
#         print(out)
#         print(out[0][ind[0]], " :: ", out[0][ind[1]])
#         print(out[0][ind_last[0]], " :: ", out[0][ind_last[1]])
#         print (int(sample['label'][0]))
#         print (ind[0])
        if int(sample['label'][0]) in ind : 
            acc += 1
#         if int(sample['label'][0]) == ind[0] or int(sample['label'][0]) == ind[1] or int(sample['label'][0]) == ind[2] or int(sample['label'][0]) == ind[3] or int(sample['label'][0]) == ind[4]:
#             acc += 1
#         writer.writerow([sample['image_name'][0], id_to_label[ind[0]] + " " + id_to_label[ind[1]] + " " 
#                          + id_to_label[ind[2]] + " " + id_to_label[ind[3]] + " " + id_to_label[ind[4]]])


In [ ]:
print(acc, len(validation_loader))
print ("Accuracy: %f" % (acc/len(validation_loader)*100))

In [ ]:
# write to the kaggle submission file 

print("# test files :: ", len(test_loader))
writeFile = open('submission.csv', 'w')
writer = csv.writer(writeFile)
writer.writerow(["Image", "Id"])

lines = [["Image", "Id"]]

model_1.eval()
counter = 0
acc = 0

with torch.no_grad():
    for sample in test_loader:
        images = sample['image'].cuda()
#         print (sample['image_name'][0])
#         print (counter)
        counter += 1
        out = model_1(images)
        out = nn.functional.softmax(out)
        out = out.cpu().detach().numpy()
        ind = np.argsort(out)[0][-5:]
#         print(ind)
#         print (int(sample['label'][0]))
#         print (ind[0])
#         if int(sample['label'][0]) == ind[0] or int(sample['label'][0]) == ind[1] or int(sample['label'][0]) == ind[2] or int(sample['label'][0]) == ind[3] or int(sample['label'][0]) == ind[4]:
#             acc += 1
        row = [sample['image_name'][0], id_to_label[ind[-1]] + " " + id_to_label[ind[-2]] + " " 
                         + id_to_label[ind[-3]] + " " + id_to_label[ind[-4]] + " " + id_to_label[ind[-5]]]
        lines.append(row)
        writer.writerow([sample['image_name'][0], id_to_label[ind[0]] + " " + id_to_label[ind[1]] + " " 
                         + id_to_label[ind[2]] + " " + id_to_label[ind[3]] + " " + id_to_label[ind[4]]])
    
print("# counter:: ", counter)

with open('submission_alexnet.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(lines)

#check how many rows are present in the file.
with open("submission.csv") as f:
    print("#rows in submission file :: ",sum(1 for line in f))
    
with open("submission_alexnet.csv") as f:
    print("#rows in submission_alexnet file :: ",sum(1 for line in f))

In [1]:
!nvidia-smi 

Sun May  5 18:27:20 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.72       Driver Version: 410.72       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K40c          Off  | 00000000:83:00.0 Off |                    0 |
| 23%   40C    P0    63W / 235W |      0MiB / 11441MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  